# Data Cleaning


## Objectives

+ Imput missing data.
+ Clean and engineer data.

## Inputs

+ outputs/datasets/collection/LoanDefault.csv
+ outputs/datasets/cleaned/TrainSet.csv
+ outputs/datasets/cleaned/TestSet.csv

## Outputs

+ Generate a cleaned dataset and divide it into train and test sets and save them in outputs/datasets/cleaned.

---

## Preparing for cleaning

We must first change the working directory and then load the data.


In [ ]:
import os

current_dir = os.getcwd() # get the current working directory
current_dir

In [ ]:
os.chdir(os.path.dirname(current_dir)) # change directory to parent directory
print("The directory you are in is:", os.getcwd()) # print current directory

In [ ]:
import pandas as pd

df = pd.read_csv("outputs/datasets/collection/LoanDefault.csv")
df.isna().sum()[df.isna().sum() > 0]

We confirm that several features contain missing values. To handle them, we will apply imputation strategies based on the characteristics of each feature: numerical features will be imputed using either the mean or median, depending on their skewness; categorical features will be imputed with the most frequent category. Additionally, we will calculate the Loan-to-Value (LTV) ratio, remove outliers from this variable, and include it in the final dataset.

In [ ]:
features_with_missing = df.columns[df.isna().any()].tolist()
features_with_missing

The dataset also includes columns such as ID and Year, which do not add value to the analysis. The ID column is simply a unique identifier with no predictive power, and the Year column contains only a single value (2019), offering no variability for modeling.

In [ ]:
from feature_engine.imputation import (CategoricalImputer,
                                       MeanMedianImputer,
                                       ArbitraryNumberImputer)
from feature_engine.selection import DropFeatures
from feature_engine.outliers import OutlierTrimmer


# Drop ID and year columns
drop_features = DropFeatures(features_to_drop=['ID', 'year'])
df_cleaned = drop_features.fit_transform(df)

# Impute missing values for categorical features
impute_categorical = CategoricalImputer(imputation_method="frequent")
df_cleaned = impute_categorical.fit_transform(df_cleaned)

# Impute missing values for numerical features
median_variables = ["Upfront_charges",
                    "rate_of_interest",
                    "property_value",
                    "income"]
impute_median = MeanMedianImputer(imputation_method="median",
                                  variables=median_variables)
df_cleaned = impute_median.fit_transform(df_cleaned)

mean_variables = ["Interest_rate_spread", "dtir1"]
impute_mean = MeanMedianImputer(imputation_method="mean",
                                variables=mean_variables)
df_cleaned = impute_mean.fit_transform(df_cleaned)

number_of_terms = df["term"].max()
impute_max = ArbitraryNumberImputer(arbitrary_number=number_of_terms,
                                    variables=["term"])
df_cleaned = impute_max.fit_transform(df_cleaned)

df_cleaned.loc[df_cleaned["LTV"].isnull(), "LTV"] = (
    df_cleaned["loan_amount"] / df_cleaned["property_value"] * 100
)

# Eliminate outliers from LTV as they don't make sense
outliers = OutlierTrimmer(capping_method="quantiles",
                          fold=0.05,
                          variables=["LTV"])
df_cleaned = outliers.fit_transform(df_cleaned)


In [ ]:
# Check for missing values again
df_cleaned.isna().sum()[df_cleaned.isna().sum() > 0]

### Split Train and Test Set

In [ ]:
from sklearn.model_selection import train_test_split
TrainSet, TestSet, _, __ = train_test_split(
                                        df_cleaned,
                                        df_cleaned['Status'],
                                        test_size=0.2,
                                        random_state=0)

print(f"TrainSet shape: {TrainSet.shape} \nTestSet shape: {TestSet.shape}")

### Save the data

In [ ]:
import os
try: 
    os.makedirs("outputs/datasets/cleaned")
except Exception as e:
    print(e)

In [ ]:
TrainSet.to_csv("outputs/datasets/cleaned/TrainSet.csv", index=False)
TestSet.to_csv("outputs/datasets/cleaned/TestSet.csv", index=False)

### Data engineering

Using these custom functions provided by Code Institute to assess whether applying numerical transformations could help make the numerical features more normally distributed.

In [ ]:
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import warnings
from feature_engine import transformation as vt
from feature_engine.outliers import Winsorizer
from feature_engine.encoding import OrdinalEncoder
warnings.filterwarnings('ignore')


def FeatureEngineeringAnalysis(df, analysis_type=None):
    """
    - used for quick feature engineering on numerical and categorical variables
    to decide which transformation can better transform the distribution shape
    - Once transformed, use a reporting tool, like ydata-profiling, to evaluate distributions
    """
    check_missing_values(df)
    allowed_types = ['numerical', 'ordinal_encoder', 'outlier_winsorizer']
    check_user_entry_on_analysis_type(analysis_type, allowed_types)
    list_column_transformers = define_list_column_transformers(analysis_type)

    # Loop in each variable and engineer the data according to the analysis type
    df_feat_eng = pd.DataFrame([])
    for column in df.columns:
        # create additional columns (column_method) to apply the methods
        df_feat_eng = pd.concat([df_feat_eng, df[column]], axis=1)
        for method in list_column_transformers:
            df_feat_eng[f"{column}_{method}"] = df[column]

        # Apply transformers in respective column_transformers
        df_feat_eng, list_applied_transformers = apply_transformers(
            analysis_type, df_feat_eng, column)

        # For each variable, assess how the transformations perform
        transformer_evaluation(
            column, list_applied_transformers, analysis_type, df_feat_eng)

    return df_feat_eng


def check_user_entry_on_analysis_type(analysis_type, allowed_types):
    """ Check analysis type """
    if analysis_type is None:
        raise SystemExit(
            f"You should pass analysis_type parameter as one of the following options: {allowed_types}")
    if analysis_type not in allowed_types:
        raise SystemExit(
            f"analysis_type argument should be one of these options: {allowed_types}")


def check_missing_values(df):
    if df.isna().sum().sum() != 0:
        raise SystemExit(
            f"There is a missing value in your dataset. Please handle that before getting into feature engineering.")


def define_list_column_transformers(analysis_type):
    """ Set suffix columns according to analysis_type"""
    if analysis_type == 'numerical':
        list_column_transformers = [
            "log_e", "log_10", "reciprocal", "power", "box_cox", "yeo_johnson"]

    elif analysis_type == 'ordinal_encoder':
        list_column_transformers = ["ordinal_encoder"]

    elif analysis_type == 'outlier_winsorizer':
        list_column_transformers = ['iqr']

    return list_column_transformers


def apply_transformers(analysis_type, df_feat_eng, column):
    for col in df_feat_eng.select_dtypes(include='category').columns:
        df_feat_eng[col] = df_feat_eng[col].astype('object')

    if analysis_type == 'numerical':
        df_feat_eng, list_applied_transformers = FeatEngineering_Numerical(
            df_feat_eng, column)

    elif analysis_type == 'outlier_winsorizer':
        df_feat_eng, list_applied_transformers = FeatEngineering_OutlierWinsorizer(
            df_feat_eng, column)

    elif analysis_type == 'ordinal_encoder':
        df_feat_eng, list_applied_transformers = FeatEngineering_CategoricalEncoder(
            df_feat_eng, column)

    return df_feat_eng, list_applied_transformers


def transformer_evaluation(column, list_applied_transformers, analysis_type, df_feat_eng):
    # For each variable, assess how the transformations perform
    print(f"* Variable Analyzed: {column}")
    print(f"* Applied transformation: {list_applied_transformers} \n")
    for col in [column] + list_applied_transformers:

        if analysis_type != 'ordinal_encoder':
            DiagnosticPlots_Numerical(df_feat_eng, col)

        else:
            if col == column:
                DiagnosticPlots_Categories(df_feat_eng, col)
            else:
                DiagnosticPlots_Numerical(df_feat_eng, col)

        print("\n")


def DiagnosticPlots_Categories(df_feat_eng, col):
    plt.figure(figsize=(4, 3))
    sns.countplot(data=df_feat_eng, x=col, palette=[
                  '#432371'], order=df_feat_eng[col].value_counts().index)
    plt.xticks(rotation=90)
    plt.suptitle(f"{col}", fontsize=30, y=1.05)
    plt.show()
    print("\n")


def DiagnosticPlots_Numerical(df, variable):
    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    sns.histplot(data=df, x=variable, kde=True, element="step", ax=axes[0])
    stats.probplot(df[variable], dist="norm", plot=axes[1])
    sns.boxplot(x=df[variable], ax=axes[2])

    axes[0].set_title('Histogram')
    axes[1].set_title('QQ Plot')
    axes[2].set_title('Boxplot')
    fig.suptitle(f"{variable}", fontsize=30, y=1.05)
    plt.tight_layout()
    plt.show()


def FeatEngineering_CategoricalEncoder(df_feat_eng, column):
    list_methods_worked = []
    try:
        encoder = OrdinalEncoder(encoding_method='arbitrary', variables=[
                                 f"{column}_ordinal_encoder"])
        df_feat_eng = encoder.fit_transform(df_feat_eng)
        list_methods_worked.append(f"{column}_ordinal_encoder")

    except Exception:
        df_feat_eng.drop([f"{column}_ordinal_encoder"], axis=1, inplace=True)

    return df_feat_eng, list_methods_worked


def FeatEngineering_OutlierWinsorizer(df_feat_eng, column):
    list_methods_worked = []

    # Winsorizer iqr
    try:
        disc = Winsorizer(
            capping_method='iqr', tail='both', fold=1.5, variables=[f"{column}_iqr"])
        df_feat_eng = disc.fit_transform(df_feat_eng)
        list_methods_worked.append(f"{column}_iqr")
    except Exception:
        df_feat_eng.drop([f"{column}_iqr"], axis=1, inplace=True)

    return df_feat_eng, list_methods_worked


def FeatEngineering_Numerical(df_feat_eng, column):
    list_methods_worked = []

    # LogTransformer base e
    try:
        lt = vt.LogTransformer(variables=[f"{column}_log_e"])
        df_feat_eng = lt.fit_transform(df_feat_eng)
        list_methods_worked.append(f"{column}_log_e")
    except Exception:
        df_feat_eng.drop([f"{column}_log_e"], axis=1, inplace=True)

    # LogTransformer base 10
    try:
        lt = vt.LogTransformer(variables=[f"{column}_log_10"], base='10')
        df_feat_eng = lt.fit_transform(df_feat_eng)
        list_methods_worked.append(f"{column}_log_10")
    except Exception:
        df_feat_eng.drop([f"{column}_log_10"], axis=1, inplace=True)

    # ReciprocalTransformer
    try:
        rt = vt.ReciprocalTransformer(variables=[f"{column}_reciprocal"])
        df_feat_eng = rt.fit_transform(df_feat_eng)
        list_methods_worked.append(f"{column}_reciprocal")
    except Exception:
        df_feat_eng.drop([f"{column}_reciprocal"], axis=1, inplace=True)

    # PowerTransformer
    try:
        pt = vt.PowerTransformer(variables=[f"{column}_power"])
        df_feat_eng = pt.fit_transform(df_feat_eng)
        list_methods_worked.append(f"{column}_power")
    except Exception:
        df_feat_eng.drop([f"{column}_power"], axis=1, inplace=True)

    # BoxCoxTransformer
    try:
        bct = vt.BoxCoxTransformer(variables=[f"{column}_box_cox"])
        df_feat_eng = bct.fit_transform(df_feat_eng)
        list_methods_worked.append(f"{column}_box_cox")
    except Exception:
        df_feat_eng.drop([f"{column}_box_cox"], axis=1, inplace=True)

    # YeoJohnsonTransformer
    try:
        yjt = vt.YeoJohnsonTransformer(variables=[f"{column}_yeo_johnson"])
        df_feat_eng = yjt.fit_transform(df_feat_eng)
        list_methods_worked.append(f"{column}_yeo_johnson")
    except Exception:
        df_feat_eng.drop([f"{column}_yeo_johnson"], axis=1, inplace=True)

    return df_feat_eng, list_methods_worked

Categorical features will be converted to numerical values using an Ordinal Encoder. Before applying the transformation, we separate the categorical and numerical variables to preserve the original categories, ensuring consistent transformation.

In [ ]:
categorical_variables = TrainSet.select_dtypes(include='object').columns.to_list()
numerical_variables = TrainSet.select_dtypes(exclude='object').columns.to_list()

In [ ]:
from feature_engine.encoding import OrdinalEncoder

encoder = OrdinalEncoder(encoding_method="arbitrary", variables=categorical_variables)
TrainSet = encoder.fit_transform(TrainSet)
TestSet = encoder.transform(TestSet)

Confirming that only the original numerical variables are being evaluated for a possible transformation.

In [ ]:
numerical_variables

In [ ]:
df_engineered = FeatureEngineeringAnalysis(
    df=TrainSet[numerical_variables],
    analysis_type='numerical'
)

After analyzing the custom transformation functions provided by Code Institute, we observed that the Q-Q plots for `loan_amount` and `property_value` improved with a logarithmic transformation, while `rate_of_interest` showed better normality with a Yeo-Johnson transformation.

In [ ]:
var_to_log = ["loan_amount", "property_value"]
var_to_yeo = ["rate_of_interest"]

log_transf = vt.LogTransformer(variables=var_to_log)
TrainSet = log_transf.fit_transform(TrainSet)
TestSet = log_transf.transform(TestSet)

yeo_transf = vt.YeoJohnsonTransformer(variables=var_to_yeo)
TrainSet = yeo_transf.fit_transform(TrainSet)
TestSet = yeo_transf.transform(TestSet)

These are the categorical variables: `['loan_limit','Gender', 'approv_in_adv', 'loan_type', 'loan_purpose', 'Credit_Worthiness', 'open_credit', 'business_or_commercial', 'Neg_ammortization', 'interest_only', 'lump_sum_payment', 'construction_type', 'occupancy_type', 'Secured_by', 'total_units', 'credit_type' 'co-applicant_credit_type', 'age', 'submission_of_application','Region','Security_Type']`

And the numerical: `['loan_amount', 'property_values]` to apply log and `['rate_of_interest']` to apply yeo-johnson.